# SurvivalLCS Experiment Runs

## Import and Setup

### Load packages

In [1]:
import os
import pandas as pd
import numpy as np
import random
import sys
import glob
from datetime import date
import argparse
from random import shuffle
from random import sample
import matplotlib.pyplot as plt
import sys
import shutil
import sksurv
import pickle
from sklearn.impute import KNNImputer
from sklearn.impute import SimpleImputer
from survival_LCS_pipeline import survivalLCS

pygame 2.5.2 (SDL 2.28.2, Python 3.9.19)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [2]:
sys.path.append("/home/bandheyh/common/survival-lcs")

In [3]:
plt.ioff()
plt.ioff()

### Import py scripts

In [4]:
import survival_AttributeTracking
import survival_Classifier
import survival_ClassifierSet
import survival_DataManagement
import survival_ExpertKnowledge
import survival_ExSTraCS
import survival_IterationRecord
import survival_Pareto
import survival_Prediction
import survival_StringEnumerator
import survival_OfflineEnvironment
import survival_Timer
import survival_RuleCompaction
import survival_Metrics
import utils
import nonparametric_estimators
import importGametes
import survival_data_simulator

### Test run scripts interactively

In [5]:
%run -i survival_AttributeTracking.py
%run -i survival_Classifier.py
%run -i survival_ClassifierSet.py
%run -i survival_DataManagement.py
%run -i survival_ExpertKnowledge.py
%run -i survival_ExSTraCS.py
%run -i survival_IterationRecord.py
%run -i survival_Pareto.py
%run -i survival_Prediction.py
%run -i survival_StringEnumerator.py
%run -i survival_OfflineEnvironment.py
%run -i survival_Timer.py
%run -i survival_RuleCompaction.py
%run -i survival_Metrics.py
%run -i utils.py
%run -i nonparametric_estimators.py

## Survival-LCS Parameters

### Set file names and necessary parameters

In [6]:
# parameter to run using hpc resources
HPC = True

homedir = "/home/bandheyh/common/survival-lcs/pipeline_3"
models = ['me', 'epi', 'het', 'add']
m0s = []

c = [0.1,0.4,0.8]
nfeat = ['f100','f1000', 'f10000'] #add f10000 when on cluster
maf = ['maf0.2','maf0.4']

iterations = 50000
cv_splits = 5

DEBUG = False
if DEBUG:
    models = ['me']
    c = [0.1]
    nfeat = ['f100', 'f1000']
    maf = ['maf0.2', 'maf0.4']
    iterations = 1000
    cv_splits = 3

### Create empty brier score DataFrame
brier_df = pd.DataFrame()
cox_brier_df = pd.DataFrame()

# other non-parameters

simulated = True # CHANGE THIS TO FALSE IF RUNNING REAL DATA

lcs_run = True
dtype_list = []

### Import the survival_LCS pipeline

In [7]:
from survival_LCS_pipeline import survivalLCS

### Making the directory structure

You'll need to create the following folders and subfolders, INSIDE of the home directory for output files:
1. cv_sim_data (with subfolders: cv_me, cv_epi, cv_het, cv_add)
2. pickled_cv_models (with subfolders: me, epi, het, add)
3. sim_lcs_output (with subfolders: me, epi, het, add)

In [8]:
def make_folder(path, overwrite=False):
    if not os.path.exists(path):
        os.makedirs(path)
    else:
        if overwrite:
            shutil.rmtree(path)
            os.makedirs(path)

In [9]:
def make_folder_structure(homedir, models, overwrite=True):
    if overwrite==True:
        make_folder(homedir+'/cv_sim_data/')
        make_folder(homedir+'/pickled_cv_models/')
        make_folder(homedir+'/sim_lcs_output/')
        for model in models:
            make_folder(homedir+'/cv_sim_data/cv_' + model, overwrite=overwrite)
            make_folder(homedir+'/pickled_cv_models/' + model, overwrite=overwrite)
            make_folder(homedir+'/sim_lcs_output/' + model, overwrite=overwrite)
    else:
        raise NotImplemented

In [10]:
make_folder_structure(homedir, models)

### Run the survival_LCS pipeline

In [11]:
def get_parameters(models, nfeat, maf, i, j, k):

    g = homedir + '/' + 'simulated_datasets/' + \
        'EDM-1_one_of_each/'+str(models[i]) + \
        '_' + str(nfeat[j]) + '_' + str(maf[k]) + '_' + 'EDM-1_01.txt'
    dtype = str(models[i]) + '_' + str(nfeat[j]) + '_' + str(maf[k])
    dtype_list.append(dtype)
    print(g)

    d = homedir + '/' + 'cv_sim_data/cv_' + str(models[i]) + '/' + dtype
    m = homedir + '/' + 'pickled_cv_models/' + str(models[i]) + '/' + dtype
    o = homedir + '/' + 'sim_lcs_output/' + str(models[i]) + '/' + dtype

    ### Set m0_path
    if models[i] in ['me','add','het']:
        m0_path = homedir+'/'+'simulated_datasets/'+'EDM-1_one_of_each/model_files/me_h0.2_'+str(maf[k])+'_Models.txt'
    else:
        m0_path = homedir+'/'+'simulated_datasets/'+'EDM-1_one_of_each/model_files/epi_h0.2_'+str(maf[k])+'_Models.txt'

    ### Set m1_path
    if models[i] in ['me','epi']:
        m1_path = None
    else:
        m1_path = homedir+'/'+'simulated_datasets/'+'EDM-1_one_of_each/model_files/epi_h0.2_'+str(maf[k])+'_Models.txt'

    ### Set m0_type
    if models[i] in ['me','add','het']:
        m0_type = 'main_effect'
    else:
        m0_type = '2way_epistasis'

    ### Set m1_type
    if models[i] in ['me', 'epi']:
        m1_type = None
    else:
        m1_type = '2way_epistasis'

    ### Set mtype
    if models[i] == 'me':
        mtype = 'main_effect'
    elif models[i] == 'epi':
        mtype = '2way_epistasis'
    elif models[i] == 'add':
        mtype = 'additive'
    else:
        mtype = 'heterogeneous'


    e = "testallsims"
    print(str(models[i])+'_'+str(nfeat[j])+'_'+str(maf[k]))

    return g, mtype, d, m, o, e,brier_df,cox_brier_df, m0_path, m0_type, m1_path, m1_type



In [12]:
def run_slcs(survivalLCS):
    survivalLCS.returnPenetrance()
    survivalLCS.returnSurvivalData()

    lcs_run = True

    if lcs_run == True:
        survivalLCS.returnCVDatasets()
        survivalLCS.returnCVModelFiles()

        current_ibs = survivalLCS.returnIBSresults()
        # current_ibs = current_ibs.rename(columns={"mean": str(models[i])+'_'+str(nfeat[j])+'_'+str(maf[k]), 
        #                                           "ci_lower": str(models[i])+'_'+str(nfeat[j])+'_'+str(maf[k])+'_ci_lower', 
        #                                           "ci_upper": str(models[i])+'_'+str(nfeat[j])+'_'+str(maf[k])+'_ci_upper'})
    else:
        print("Datasets generated only")

    print(survivalLCS.model_type)

    return current_ibs

In [13]:
def make_breir_output(brier_df_list, output_path, model_type, models, dtype_list, i):
    brier_df = pd.concat(brier_df_list, axis = 1, sort = False).reset_index()

    brier_df.to_csv(homedir +'/'+'sim_lcs_output/'+str(models[i])+'/ibs_data_'+mtype+'.txt', index = False)

    plt.figure(figsize=(10, 10))
    plt.xlabel('Time')
    plt.ylabel('Brier score')
    plt.ylim(0,1)

    for i in range(1,len(dtype_list)):
        plt.plot(brier['times'], brier[dtype_list[i]],label = brier[dtype_list[i]].name)
        plt.fill_between(brier['times'], brier[dtype_list[i]+'_ci_lower'], brier[dtype_list[i]+'_ci_upper'], color='b', alpha=.1)
    plt.savefig(output_path+'/brier_scores_'+model_type + '.png')

In [14]:
%%capture
from survival_LCS_pipeline import survivalLCS
job_obj_list = list()
for i in range(0,len(models)):
    for j in range(0,len(nfeat)):
        brier_df_list = list()
        for k in range(0,len(maf)):
            g, mtype, d, m, o, e,brier_df,cox_brier_df, m0_path, m0_type, m1_path, m1_type = get_parameters(models, nfeat, maf, i, j, k)
            slcs = survivalLCS(g, mtype, d, m, o, e,brier_df,cox_brier_df, m0_path, m0_type, m1_path, m1_type, 
                                      c = c,iterations = iterations, cv = cv_splits)
            if HPC == False:
                current_ibs = run_slcs(slcs)
                brier_df_list.append(current_ibs)
            else:
                job_obj_list.append(slcs)
        if HPC == False:
            if lcs_run == True:
                make_breir_output(brier_df_list, survivalLCS.output_path, survivalLCS.model_type, models, dtype_list, i)
            else:
                print('LCS not run, no brier scores available')

## HPC Code

In [15]:
import dask
from dask.distributed import Client
from dask_jobqueue import SLURMCluster, LSFCluster, SGECluster

In [16]:
def get_cluster(cluster_type='SLURM', output_path=".", queue='defq', memory=16):
    client = None
    try:
        if cluster_type == 'SLURM':
            cluster = SLURMCluster(queue=queue,
                                   cores=1,
                                   memory=str(memory) + "G",
                                   walltime="24:00:00",
                                   log_directory=output_path + "/dask_logs/")
            cluster.adapt(maximum_jobs=400)
        elif cluster_type == "LSF":
            cluster = LSFCluster(queue=queue,
                                 cores=1,
                                 mem=memory * 1000000000,
                                 memory=str(memory) + "G",
                                 walltime="24:00",
                                 log_directory=output_path + "/dask_logs/")
            cluster.adapt(maximum_jobs=400)
        elif cluster_type == 'UGE':
            cluster = SGECluster(queue=queue,
                                 cores=1,
                                 memory=str(memory) + "G",
                                 resource_spec="mem_free=" + str(memory) + "G",
                                 walltime="24:00:00",
                                 log_directory=output_path + "/dask_logs/")
            cluster.adapt(maximum_jobs=400)
        elif cluster_type == 'Local':
            c = Client()
            cluster = c.cluster
        else:
            raise Exception("Unknown or Unsupported Cluster Type")
        client = Client(cluster)
    except Exception as e:
        print(e)
        raise Exception("Exception: Unknown Exception")
    print("Running dask-cluster")
    print(client.scheduler_info())
    return client

In [17]:
cluster = get_cluster(output_path=homedir)

Running dask-cluster
{'type': 'Scheduler', 'id': 'Scheduler-4ed9f1fb-719f-4e6b-8a01-82d94228c28a', 'address': 'tcp://10.17.134.112:37141', 'services': {'dashboard': 46409}, 'started': 1712477423.974513, 'workers': {}}


/home/bandheyh/common/anaconda3/envs/slcs/lib/python3.9/site-packages/distributed/node.py:182: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 46409 instead
  warnings.warn(


In [18]:
make_folder(homedir+'/dask_logs/', overwrite=True)

In [19]:
def run_parallel(model):
    try:
        brier_df = run_slcs(model)
    except Exception as e:
        raise e
        brier_df = e
    return brier_df

In [20]:
job_obj_list

In [21]:
if HPC == True:
    delayed_results = []
    for model in job_obj_list:
        brier_df = dask.delayed(run_parallel)(model)
        delayed_results.append(brier_df)
    results = dask.compute(*delayed_results)

In [ ]:
# if HPC:
#     results = dask.compute([dask.delayed(run_parallel)(model) for model in job_obj_list])

In [ ]:
if HPC:
    with open(homedir+'/results.pkl', 'wb') as file:
        pickle.dump(results, file, pickle.HIGHEST_PROTOCOL)

### Error Checking

In [ ]:
error_idxs = list()
for i in range(len(results)):
    if type(results[i]) ==  ValueError:
        print(i, results[i])
        error_idxs.append(i)


In [ ]:
arr = np.arange(len(results)).reshape(len(models), len(nfeat), len(maf))

# Convert a 1D index to a 3D index
for x in error_idxs:
    i, j, k = np.unravel_index(x, arr.shape)
    print(models[i], nfeat[j], maf[k])

In [ ]:
brier_df_list = list()
arr = np.arange(len(results)).reshape(len(models), len(nfeat), len(maf))
for x in range(len(results)):
    i, j, k = np.unravel_index(x, arr.shape)
    print(models[i], nfeat[j], maf[k])
    current_ibs = results[x]
    current_ibs = current_ibs.rename(columns={"mean": str(models[i])+'_'+str(nfeat[j])+'_'+str(maf[k]), 
                                            "ci_lower": str(models[i])+'_'+str(nfeat[j])+'_'+str(maf[k])+'_ci_lower', 
                                            "ci_upper": str(models[i])+'_'+str(nfeat[j])+'_'+str(maf[k])+'_ci_upper'})
    brier_df_list.append(current_ibs)
brier_df = pd.concat(brier_df_list, axis = 1, sort = False).reset_index()
#print('brier_df:', brier_df)
brier_df.to_csv(homedir+'/ibs_data_all.csv', index = False)
brier_df

me f100 maf0.2
me f100 maf0.4
me f1000 maf0.2
me f1000 maf0.4


,times,me_f100_maf0.2_cens0.1,me_f100_maf0.2_cens0.1_ci_lower,me_f100_maf0.2_cens0.1_ci_upper,me_f100_maf0.4_cens0.1,me_f100_maf0.4_cens0.1_ci_lower,me_f100_maf0.4_cens0.1_ci_upper,me_f1000_maf0.2_cens0.1,me_f1000_maf0.2_cens0.1_ci_lower,me_f1000_maf0.2_cens0.1_ci_upper,me_f1000_maf0.4_cens0.1,me_f1000_maf0.4_cens0.1_ci_lower,me_f1000_maf0.4_cens0.1_ci_upper
0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1.0,0.800388,0.783085,0.822725,0.052230,0.041947,0.065506,0.737541,0.670840,0.823652,0.001504,-0.001504,0.005386
2,2.0,0.520806,0.500918,0.546481,0.070483,0.054784,0.090750,0.485643,0.447042,0.535477,0.001504,-0.001504,0.005386
3,3.0,0.354661,0.331700,0.384304,0.083654,0.064721,0.108096,0.334093,0.313524,0.360648,0.001504,-0.001504,0.005386
4,4.0,0.283756,0.257274,0.317944,0.093769,0.077480,0.114799,0.269525,0.257410,0.285165,0.001503,-0.001503,0.005385
...,...,...,...,...,...,...,...,...,...,...,...,...,...
96,96.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
97,97.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
98,98.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
99,99.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
